In [1]:
import pandas as pd

from typing import Any, Optional

import pandas as pd
from visions import VisionsTypeset
import re
import os
from datetime import datetime,timedelta

In [40]:
full_path = os.getcwd()
customers = pd.read_csv(full_path.partition('notebooks')[0] + '/data/01_raw/Customers.csv')
loans = pd.read_csv(full_path.partition('notebooks')[0] + '/data/01_raw/Loans_20240131.csv')
funds = pd.read_csv(full_path.partition('notebooks')[0] + '/data/01_raw/Funds_Hist_to_20240331.csv')
transactions = pd.read_csv(full_path.partition('notebooks')[0] + '/data/01_raw/Transactions_to_20240331.csv')
loans_hist = pd.read_csv(full_path.partition('notebooks')[0] + '/data/01_raw/Loans_Hist_to_20240331.csv')

/var/folders/s7/s9wfpwyx2dj_9c9q_c07v03w0000gn/T/ipykernel_62545/3263252385.py:6: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  loans_hist = pd.read_csv(full_path.partition('notebooks')[0] + '/data/01_raw/Loans_Hist_to_20240331.csv')


## 1. Feature Enginnering Transactions dataset

Since we are using the snapshot from 2025-03-31 (Loans_20240331.csv), which includes all loans granted between 2024-03-01 and 2024-03-31, we will analyse the loan defaults approximately one year after their creation (that why the snapshot is 2025-03-31)

To engineer features from the transactions dataset, such as average monthly expenses, we will consider only the transactions that occurred before 2023-12-31(one month before the loan creation). 

The following features will be created per customer:

__`Avg_Monthly_Income`__: The average monthly inflows (e.g., salary, deposits) into the customer's account.

__`Income_Stability`__: The standard deviation of monthly income, which reflects how stable or volatile the customer's income is over time.

__`Avg_Monthly_Expenses`__: The average monthly outflows (e.g., purchases, bill payments) from the customer's account.

__`Expenses_Stability`__: The standard deviation of monthly expenses, which helps us understand whether the customer's spending habits are consistent or erratic.

This ensures that we are using only historical data that would have been available prior to the loan issuance, avoiding data leakage.


- Let's define Loans reference Date =  2024-01-31 

In [41]:
loans_reference_date=pd.to_datetime('2024-01-31')

In [42]:
end_date=loans_reference_date-pd.DateOffset(months=1)
start_date = end_date - pd.DateOffset(years=1) + pd.DateOffset(days=1)

In [243]:
end_date

Timestamp('2023-12-31 00:00:00')

- Filtering the transactions dataset

In [244]:
# Ensure 'Date' is datetime
transactions["Date"] = pd.to_datetime(transactions["Date"])
# Filter the transactions
transactions = transactions[
    (transactions["Date"] >= start_date) &
    (transactions["Date"] <= end_date)
].copy()

- Transform CustomerIdDebit and CustomerIdCredit to int

In [245]:
transactions["CustomerIdCreditNew"]=transactions["CustomerIdCreditNew"].fillna(0).astype(int)
transactions["CustomerIdDebitNew"]=transactions["CustomerIdDebitNew"].fillna(0).astype(int)

In [246]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6470399 entries, 0 to 6470398
Data columns (total 9 columns):
 #   Column               Dtype         
---  ------               -----         
 0   Date                 datetime64[ns]
 1   TransactionId        int64         
 2   CustomerIdDebitNew   int32         
 3   CustomerIdCreditNew  int32         
 4   TransactionType      object        
 5   TransactionCategory  object        
 6   Currency             object        
 7   Amount               float64       
 8   AmountMZN            float64       
dtypes: datetime64[ns](1), float64(2), int32(2), int64(1), object(3)
memory usage: 444.3+ MB


In [247]:

# remove transactions not related to the customers
transactions = transactions[~((transactions.CustomerIdCreditNew==0) & (transactions.CustomerIdDebitNew==0))]

In [248]:
transactions

,Date,TransactionId,CustomerIdDebitNew,CustomerIdCreditNew,TransactionType,TransactionCategory,Currency,Amount,AmountMZN
2,2023-01-03,3,111024,24625,D,IT,ZAR,27524.86,103686.15
3,2023-01-03,4,82540,0,D,ACAW,MZN,500.00,500.00
5,2023-01-03,6,21668,0,D,ACPS,MZN,7644.33,7644.33
6,2023-01-03,7,104307,0,D,ACAW,MZN,2000.00,2000.00
9,2023-01-03,10,76849,0,D,ACAT,MZN,20000.00,20000.00
...,...,...,...,...,...,...,...,...,...
6470393,2023-12-29,6470394,111024,47928,D,IT,ZAR,567000.00,1952748.00
6470394,2023-12-29,6470395,32921,83227,D,IT,USD,4490.00,286803.24
6470395,2023-12-29,6470396,111024,68954,D,IT,ZAR,2034022.10,6936015.36
6470396,2023-12-29,6470397,32892,22127,D,IT,EUR,1705.00,119179.50


- Transform the date to date so that we can get the month

In [249]:
transactions['Month']=transactions["Date"].dt.to_period('M')

- Grouping and summing monthly incomes and expenses per customer 

In [250]:
transactions_cus_debited=transactions[['Month', 'CustomerIdDebitNew', 'AmountMZN']].groupby(['Month', 'CustomerIdDebitNew']).sum(numeric_only=True).reset_index().rename(columns={'AmountMZN': 'Monthly_Expenses','CustomerIdDebitNew':'CustomerId'})
transactions_cus_credited=transactions[['Month', 'CustomerIdCreditNew', 'AmountMZN']].groupby(['Month', 'CustomerIdCreditNew']).sum(numeric_only=True).reset_index().rename(columns={'AmountMZN': 'Monthly_Income','CustomerIdCreditNew':'CustomerId' })
transactions_cus_debited=transactions_cus_debited[transactions_cus_debited["CustomerId"]!=0]
transactions_cus_credited=transactions_cus_credited[transactions_cus_credited["CustomerId"]!=0]

- Getting the montly average of the income and expenses per Customer

In [251]:
avg_income_per_cus=(
    transactions_cus_credited
    .groupby("CustomerId")
    .agg(Avg_Monthly_Income=('Monthly_Income','mean'),
         Income_Stability=('Monthly_Income','std'),
    )
    .reset_index()
)
avg_expenses_per_cus=(
    transactions_cus_debited
    .groupby("CustomerId")
    .agg(Avg_Monthly_expenses=('Monthly_Expenses', 'mean'),
         Expenses_Stability=('Monthly_Expenses','std')
    )
    .reset_index()
)
 

In [252]:
avg_income_per_cus.count()

CustomerId            28948
Avg_Monthly_Income    28948
Income_Stability      24626
dtype: int64

- Combine the income and expenses into customer_transactional_summary

In [253]:
customer_transactional_summary=pd.merge(
    avg_income_per_cus,
    avg_expenses_per_cus,
    on="CustomerId",
    how="outer"
)

- Fill the NAN values with 0

In [254]:
customer_transactional_summary=customer_transactional_summary.fillna(0)

In [255]:
customer_transactional_summary

,CustomerId,Avg_Monthly_Income,Income_Stability,Avg_Monthly_expenses,Expenses_Stability
0,94,4.600000e+05,160208.197876,6.119526e+05,318717.326707
1,95,2.137482e+05,52480.144834,2.196227e+05,212327.051225
2,97,1.469710e+06,602019.295499,1.816162e+06,998222.338814
3,100,5.600000e+05,0.000000,2.539250e+05,76261.466351
4,104,1.419289e+05,74736.516948,1.304371e+05,27063.162667
...,...,...,...,...,...
30373,121638,0.000000e+00,0.000000,3.696773e+04,0.000000
30374,121979,0.000000e+00,0.000000,6.107980e+03,0.000000
30375,121993,0.000000e+00,0.000000,1.390609e+04,0.000000
30376,122190,0.000000e+00,0.000000,2.000000e+05,0.000000


## 2. Feature Enginnering Funds dataset

In this step, we will engineer features from the Funds dataset, focusing only on data before 2024-01-01, to avoid using information that would not have been available at the time the loan was granted.

The features we aim to extract include:

**Avg_Monthly_Funds**: The average amount of funds (e.g., income, deposits) observed per month.

**Funds_Stability**: A metric that captures the consistency or variability of the funds over time , we will be using the standard deviation.

These features will help us assess the financial stability of customers before they received their loan, which is a key factor in predicting defaults.

- Filter the funds datset

In [256]:
funds["Date"] = pd.to_datetime(funds["Date"])

# Filter within 1-year window before loan date
funds = funds[
    (funds["Date"] >= start_date) &
    (funds["Date"] <= end_date)
].copy()
funds.sort_values(by="Date",ascending=False)

,Date,CustomerId,FundsBalance
502308,2023-12-31,26916,1.049379e+04
473491,2023-12-31,115657,3.019630e+04
473499,2023-12-31,27852,1.043095e+05
473498,2023-12-31,104460,3.648621e+04
473497,2023-12-31,52674,1.188384e+03
...,...,...,...
26780,2023-01-31,93476,1.396000e+02
26779,2023-01-31,47683,2.769602e+03
26778,2023-01-31,64814,1.516738e+07
26777,2023-01-31,31814,4.386710e+03


In [257]:
customer_funds_summary=(funds.groupby("CustomerId")
               .agg(Avg_Monthly_Funds=("FundsBalance","mean"),
                    Funds_Stability=("FundsBalance","std"))
                .reset_index()
                )
customer_funds_summary.head()


,CustomerId,Avg_Monthly_Funds,Funds_Stability
0,87,1.609330e+05,1.387818e+05
1,94,6.505385e+04,1.000203e+04
2,95,2.665719e+05,5.243395e+04
3,96,6.855160e+03,1.631545e+00
4,97,6.809230e+06,1.791182e+06


## 3. Feature Engineering – Loans History Dataset (Previous Loans)

In this step, we will extract features from the Loans History dataset to capture the customer’s past credit behaviour. We will only consider loans granted before 2024-01-01, as our goal is to model default risk based on information available prior to the current loan issuance.

The features to be extracted include:

- **Previous_Loans_Count**: Total number of loans the customer had before 2024-01-01.

- **Previous_Loans_Avg_Amount**: Average amount of those previous loans.

- **Previous_Loan_Std**: Standard deviation of the loan amounts, reflecting variability in the borrowed amounts.

- **Previous_Loan_Defaults**: Number of previous loans that ended in default, if available.

These features are important to understand the customer’s credit history and risk profile, which are highly predictive of future default behavior

In [258]:
loans_hist["SnapshotDate"] = pd.to_datetime(loans_hist["SnapshotDate"])

# Filter using the same date range as transactions/funds
loans_hist_filtered= loans_hist[
    #(loans_hist["Date"] >= start_date) & - we want to get all history
    (loans_hist["SnapshotDate"] <= end_date)
].copy()

loans_hist_filtered.sort_values(by="SnapshotDate", ascending=False)

,SnapshotDate,CustomerNewId,ContractId,SegmentDesc,CreditType,CreditAmount,Outstanding,CreditEOMStartDate,CreditEOMEndDate,NumberOfInstallmentsToPay,PaymentFrequency,Arreas,HasDefault
66344,2023-12-31,83438,3220014,Wholesale,Bill of Exchange Discount,10499132.88,10499132.88,12/31/2023,2/29/2024,NaN,Single,NaN,0
58242,2023-12-31,102982,2231327,Retail,Credit Card,100000.00,30437.24,6/30/2023,6/30/2026,NaN,Monthly,NaN,0
58251,2023-12-31,112008,2873645,Wholesale,Credit Card,150000.00,0.00,6/30/2023,6/30/2026,NaN,Monthly,NaN,0
58250,2023-12-31,43360,2386462,Retail,Credit Card,150000.00,56285.77,6/30/2023,6/30/2026,NaN,Monthly,NaN,0
58249,2023-12-31,77212,2873643,Retail,Credit Card,150000.00,1585.54,6/30/2023,6/30/2026,NaN,Monthly,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6759,2023-07-31,80277,1079853,Retail,Credit Card,50000.00,39928.87,3/31/2023,3/31/2026,NaN,Monthly,NaN,0
6758,2023-07-31,42357,1286709,Retail,Credit Card,50000.00,42372.40,4/30/2023,4/30/2026,NaN,Monthly,NaN,0
6757,2023-07-31,61963,1220859,Retail,Credit Card,40000.00,25868.00,3/31/2023,3/31/2026,NaN,Monthly,NaN,0
6756,2023-07-31,97161,1070403,Retail,Credit Card,90000.00,1900.00,3/31/2023,3/31/2026,NaN,Monthly,NaN,0


__`For Previous_Loans_Count, Previous_Loans_Avg_Smount, Previous_Loan_Std`__:

Since we have monthly positions of loans, and want one row per contract, we want to  get the latest or earliest snapshot per ContractId:

In [259]:
ordered_loans_hist=loans_hist_filtered.sort_values(['ContractId','SnapshotDate'], ascending=[True, False])
latest_loans=ordered_loans_hist.drop_duplicates(subset='ContractId',keep='first')


In [260]:
prev_loans_per_cust=(latest_loans.groupby("CustomerNewId").agg(
    Previous_Loan_Count=("ContractId","count"),
    Previous_Loans_Avg_Amount=("CreditAmount","mean"),
    Previous_Loans_Std=("CreditAmount","std")
)).reset_index().rename(columns={"CustomerNewId":"CustomerId"})


In [261]:
prev_loans_per_cust.head()

,CustomerId,Previous_Loan_Count,Previous_Loans_Avg_Amount,Previous_Loans_Std
0,97,2,1287574.2,1.820695e+06
1,152,4,2575000.0,3.720327e+06
2,184,2,600000.0,5.656854e+05
3,205,2,200000.0,0.000000e+00
4,217,1,350000.0,NaN


For __`Previous_Loan_Defaults`__:

To compute the number of previous loan defaults, we will use the monthly positions dataset, which tracks the status of each loan over time.

Since we want to capture how many times a customer was in default, even if they later repaid the loan, we will count every month where a default status was registered. This gives us a better sense of how often the customer has shown signs of financial distress, regardless of final repayment.

In [262]:
defaults=loans_hist_filtered[loans_hist_filtered["HasDefault"]==1]
defaults.head()

,SnapshotDate,CustomerNewId,ContractId,SegmentDesc,CreditType,CreditAmount,Outstanding,CreditEOMStartDate,CreditEOMEndDate,NumberOfInstallmentsToPay,PaymentFrequency,Arreas,HasDefault
0,2023-07-31,112740,407419,Retail,Unarranged Overdraft,2.934165e+04,4.181495e+04,11/30/2022,NaN,NaN,Single,245.0,1
6,2023-07-31,95334,930186,Retail,Unarranged Overdraft,1.046457e+05,1.317331e+05,2/28/2023,NaN,NaN,Single,166.0,1
22,2023-07-31,104090,19,Wholesale,Unarranged Overdraft,3.046097e+07,3.434031e+07,2/28/2023,NaN,NaN,Single,158.0,1
26,2023-07-31,77701,23,Retail,Unarranged Overdraft,4.227500e+02,4.589550e+03,6/30/2022,NaN,NaN,Single,420.0,1
27,2023-07-31,77701,24,Retail,Unarranged Overdraft,9.064710e+01,1.231010e+02,8/31/2022,NaN,NaN,Single,365.0,1


In [263]:
prev_loans_defaults=defaults.groupby("CustomerNewId").agg(
    Previous_Loan_Defaults=("ContractId","count")
).reset_index().rename(columns={"CustomerNewId":"CustomerId"}
                       )
prev_loans_defaults.head()

,CustomerId,Previous_Loan_Defaults
0,261,6
1,323,3
2,336,3
3,439,6
4,461,2


In [264]:
customer_prev_loans_summary=pd.merge(
    prev_loans_per_cust,
    prev_loans_defaults,
    on="CustomerId",
    how="outer"
    ).fillna(0)

In [265]:
customer_prev_loans_summary

,CustomerId,Previous_Loan_Count,Previous_Loans_Avg_Amount,Previous_Loans_Std,Previous_Loan_Defaults
0,97,2,1.287574e+06,1.820695e+06,0.0
1,152,4,2.575000e+06,3.720327e+06,0.0
2,184,2,6.000000e+05,5.656854e+05,0.0
3,205,2,2.000000e+05,0.000000e+00,0.0
4,217,1,3.500000e+05,0.000000e+00,0.0
...,...,...,...,...,...
12418,122175,3,6.666667e+04,2.886751e+04,0.0
12419,122182,2,9.007808e+04,5.667897e+04,11.0
12420,122183,1,3.714200e+02,0.000000e+00,0.0
12421,122190,1,2.000000e+05,0.000000e+00,0.0


## 4. Feature Engineering – Active Loans at the Time of New Credit

In this step, we will extract features related to other active loans that a customer had prior to applying for a new loan (between 2024-01-01 and 2024-01-31). These features help us understand the customer’s existing financial commitments at the time of taking the new loan, which is highly relevant to credit risk assessment.

We will use:
Loans_20240131:  This dataset includes only the loans created between 2024-01-01 and 2024-01-31 — these are the loans we want to predict defaults on.
Loans_Hist_to_20240331:A monthly loan history dataset that contains the state of all loans, including those created before January 2024.

__`Active_Loans_Count`__: Number of other loans that were still active as of 2023-12-31 (i.e., before the new loan was created).

__`Active_Loan_Amount_Total`__: Total outstanding balance of those active loans as of 2023-12-31.

Fist step Use the 2023-12-31 snapshot from Loans_Hist_to_20240331.

In [266]:

active_loans= loans_hist[
    loans_hist["SnapshotDate"] == end_date
].copy()

In [267]:
active_loans["CreditEOMStartDate"]=pd.to_datetime(active_loans["CreditEOMStartDate"])
active_loans.sort_values(by="CreditEOMStartDate", ascending=False).head()

,SnapshotDate,CustomerNewId,ContractId,SegmentDesc,CreditType,CreditAmount,Outstanding,CreditEOMStartDate,CreditEOMEndDate,NumberOfInstallmentsToPay,PaymentFrequency,Arreas,HasDefault
66344,2023-12-31,83438,3220014,Wholesale,Bill of Exchange Discount,10499132.88,10499132.88,2023-12-31,2/29/2024,NaN,Single,NaN,0
65860,2023-12-31,53707,3118926,Retail,Personal Credit,100000.00,100000.00,2023-12-31,12/31/2028,61.0,Monthly,NaN,0
65879,2023-12-31,3789,3118945,Retail,Personal Credit,200000.00,200000.00,2023-12-31,12/31/2028,61.0,Monthly,NaN,0
55177,2023-12-31,18129,1841697,Retail,Unarranged Overdraft,2164.23,2221.51,2023-12-31,NaN,NaN,Single,12.0,0
65880,2023-12-31,28829,3118946,Wholesale,Business Loan Account,8412670.50,8412670.50,2023-12-31,4/30/2024,1.0,Single,NaN,0


From above, we see that we get all loans created until 2023-12-31

Now lets compute for each customer the Active_Loans_Count and Active_Loan_Amount_Total(The sum of their outstanding balances)

In [268]:
customer_active_loans_summary=(active_loans.groupby("CustomerNewId").agg(
    Active_Loans_Count=("ContractId","count"),
    Active_Loan_Amount_Total=("Outstanding","sum"),
)).reset_index().rename(columns={"CustomerNewId":"CustomerId"})

In [269]:
customer_active_loans_summary


,CustomerId,Active_Loans_Count,Active_Loan_Amount_Total
0,97,1,960246.36
1,152,3,3406931.57
2,184,2,1129201.47
3,205,1,0.00
4,217,1,0.00
...,...,...,...
10103,122168,1,0.00
10104,122169,1,0.00
10105,122175,2,147155.72
10106,122182,2,38283.77


## 5. Feature Engineering – Current Loans (Target Loans)

In this step, we will extract core features from the actual loans we are trying to predict defaults on — those created between 2024-01-01 and 2024-01-31, as recorded in the Loans_20240131 dataset.

These features describe the loan terms and characteristics at the time of issuance and will serve as input variables for the model.

We will include:

__`CreditType`__	Type of credit (e.g., personal loan, car loan, mortgage, etc.)

__`CreditAmount`__	Total amount granted for the loan
Duration	Loan duration, calculated as: CreditEOMEndDate CreditEOMStartDate

__`NumberOfInstallmentsToPay`__	Total number of installments scheduled for the loan

__`PaymentFrequency`__	Frequency of repayments (e.g., monthly, quarterly)

__`HasDefault`__	Binary indicator of whether the loan defaulted (used as the target label)

In [270]:
loans["CreditEOMStartDate"]=pd.to_datetime(loans["CreditEOMStartDate"])
loans["CreditEOMEndDate"]=pd.to_datetime(loans["CreditEOMEndDate"])


- Lets start with CreditType

Before using CreditType as a feature, we will apply a filter to remove "Unarranged Overdraft" entries. These are not structured or approved loans, but rather automatic overdraft facilities triggered without prior agreement. Since our objective is to predict defaults on arranged loans, including these would introduce noise and distort the model.

Therefore, we will exclude all loans where CreditType is "Unarranged Overdraft".

In [271]:
loans["CreditType"].value_counts()

CreditType
Public Sector Employee Loan    776
Arranged Overdraft              59
Personal Credit                 44
Unarranged Overdraft            30
Credit Card                     25
Leasing                          6
Business Loan Account            5
Secured Current Account          3
Name: count, dtype: int64

In [272]:
loans_to_predict=loans[loans["CreditType"]!="Unarranged Overdraft"].copy()
loans_to_predict["CreditType"].value_counts()

CreditType
Public Sector Employee Loan    776
Arranged Overdraft              59
Personal Credit                 44
Credit Card                     25
Leasing                          6
Business Loan Account            5
Secured Current Account          3
Name: count, dtype: int64

- lets calculate duration 

In [273]:
loans_to_predict["Duration_Months"] = (
    (loans_to_predict["CreditEOMEndDate"].dt.year - loans_to_predict["CreditEOMStartDate"].dt.year) * 12 +
    (loans_to_predict["CreditEOMEndDate"].dt.month - loans_to_predict["CreditEOMStartDate"].dt.month)
)


In [274]:
loans_to_predict=loans_to_predict.rename(columns={"CustomerNewId":"CustomerId"})

In [276]:
loans_to_predict[["CustomerId","CreditType","CreditAmount","Duration_Months","NumberOfInstallmentsToPay","PaymentFrequency","HasDefault"]]

,CustomerId,CreditType,CreditAmount,Duration_Months,NumberOfInstallmentsToPay,PaymentFrequency,HasDefault
0,117007,Personal Credit,82482.92,61,62.0,Monthly,0
1,116276,Credit Card,49000.00,36,NaN,Monthly,0
2,76794,Credit Card,500000.00,36,NaN,Monthly,0
3,54604,Credit Card,350000.00,36,NaN,Monthly,0
4,49422,Personal Credit,1000000.00,13,14.0,Monthly,0
...,...,...,...,...,...,...,...
943,105633,Public Sector Employee Loan,60000.00,60,61.0,Monthly,0
944,31813,Public Sector Employee Loan,50000.00,60,61.0,Monthly,0
945,52519,Public Sector Employee Loan,100000.00,60,61.0,Monthly,0
946,118117,Public Sector Employee Loan,149000.00,60,61.0,Monthly,0
